In [0]:
# Descriptive stats
events.describe(["price"]).show()

# Hypothesis: weekday vs weekend conversion
weekday = events.withColumn("is_weekend",
    F.dayofweek("event_date").isin([1,7]))
weekday.groupBy("is_weekend", "event_type").count().show()

# Correlation
events.stat.corr("price", "conversion_rate")

# Feature engineering
features = events.withColumn("hour", F.hour("event_time")) \
    .withColumn("day_of_week", F.dayofweek("event_date")) \
    .withColumn("price_log", F.log(F.col("price")+1)) \
    .withColumn("time_since_first_view",
        F.unix_timestamp("event_time") -
        F.first("event_time").over(Window.partitionBy("user_id").orderBy("event_time")))


---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-4886487906999805>, line 2
      1 # Descriptive stats
----> 2 events.describe(["price"]).show()
      4 # Hypothesis: weekday vs weekend conversion
      5 weekday = events.withColumn("is_weekend",
      6     F.dayofweek("event_date").isin([1,7]))

NameError: name 'events' is not defined

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.window import Window

spark = SparkSession.builder.getOrCreate()


In [0]:
# Load events table
events = spark.table("silver")

# Quick sanity check
events.printSchema()# Basic describe
events.describe(["price"]).show()

# Advanced stats
events.select(
    F.count("price").alias("count"),
    F.mean("price").alias("mean"),
    F.stddev("price").alias("stddev"),
    F.expr("percentile_approx(price, 0.5)").alias("median"),
    F.expr("percentile_approx(price, array(0.25, 0.75))").alias("iqr")
).show()

events.limit(5).show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4886487906999808>, line 5
      2 events = spark.table("silver")
      4 # Quick sanity check
----> 5 events.printSchema()# Basic describe
      6 events.describe(["price"]).show()
      8 # Advanced stats

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1975, in DataFrame.printSchema(self, level)
   1973     print(self.schema.treeString(level))
   1974 else:
-> 1975     print(self.schema.treeString())

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1948, in DataFrame.schema(self)
   1945 @property
   1946 def schema(self) -> StructType:
   1947     # self._schema call will cache the schema and serialize it if it is not cached yet.
-> 1948     _schema = self._schema
   1949     if self._cached_schema_serialized is not None:
   1950         try:

F

In [0]:
events = events.withColumn(
    "is_weekend",
    F.dayofweek("event_date").isin([1, 7])  # Sun, Sat
)
events = events \
    .filter(F.col("event_time").isNotNull()) \
    .filter(F.col("price").isNotNull()) \
    .withColumn("event_date", F.to_date("event_time"))


In [0]:
events.groupBy("is_weekend", "event_type").count().orderBy("is_weekend").show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4886487906999810>, line 1
----> 1 events.groupBy("is_weekend", "event_type").count().orderBy("is_weekend").show()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NOT_BOOL",
    862             messageParameters={
   (...)
    865             },
    866         )
    868 table, _ = DataFrame(
    869     plan.ShowString(
    87

In [0]:
conversion = events.groupBy("is_weekend").agg(
    F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchases"),
    F.countDistinct("user_id").alias("users")
).withColumn(
    "conversion_rate",
    F.col("purchases") / F.col("users")
)

conversion.show()


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4886487906999811>, line 9
      1 conversion = events.groupBy("is_weekend").agg(
      2     F.sum(F.when(F.col("event_type") == "purchase", 1).otherwise(0)).alias("purchases"),
      3     F.countDistinct("user_id").alias("users")
   (...)
      6     F.col("purchases") / F.col("users")
      7 )
----> 9 conversion.show()

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySpa

In [0]:
events = events.withColumn(
    "converted",
    F.when(F.col("event_type") == "purchase", 1).otherwise(0)
)


In [0]:
user_window = Window.partitionBy("user_id").orderBy("event_time")

features = events \
    .withColumn("hour", F.hour("event_time")) \
    .withColumn("day_of_week", F.dayofweek("event_date")) \
    .withColumn("price_log", F.log(F.col("price") + 1)) \
    .withColumn(
        "time_since_first_event",
        F.unix_timestamp("event_time") -
        F.unix_timestamp(F.first("event_time").over(user_window))
    ) \
    .withColumn("event_step", F.row_number().over(user_window))


In [0]:
features.select(
    "user_id", "event_time", "event_type",
    "hour", "day_of_week",
    "price", "price_log",
    "time_since_first_event",
    "event_step",
    "converted"
).limit(10).show(truncate=False)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4886487906999814>, line 8
      1 features.select(
      2     "user_id", "event_time", "event_type",
      3     "hour", "day_of_week",
      4     "price", "price_log",
      5     "time_since_first_event",
      6     "event_step",
      7     "converted"
----> 8 ).limit(10).show(truncate=False)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861     

In [0]:
user_features = features.groupBy("user_id").agg(
    F.max("converted").alias("label"),
    F.avg("price").alias("avg_price"),
    F.max("price").alias("max_price"),
    F.count("*").alias("event_count"),
    F.max("event_step").alias("session_depth"),
    F.avg("time_since_first_event").alias("avg_time_to_action"),
    F.avg("hour").alias("avg_hour"),
    F.avg("day_of_week").alias("avg_day")
)

user_features.show(10)


---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4886487906999815>, line 12
      1 user_features = features.groupBy("user_id").agg(
      2     F.max("converted").alias("label"),
      3     F.avg("price").alias("avg_price"),
   (...)
      9     F.avg("day_of_week").alias("avg_day")
     10 )
---> 12 user_features.show(10)

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:1123, in DataFrame.show(self, n, truncate, vertical)
   1122 def show(self, n: int = 20, truncate: Union[bool, int] = True, vertical: bool = False) -> None:
-> 1123     print(self._show_string(n, truncate, vertical))

File /databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/dataframe.py:876, in DataFrame._show_string(self, n, truncate, vertical)
    859     except ValueError:
    860         raise PySparkTypeError(
    861             errorClass="NO